In [1]:
# Load required libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import math
import scipy.integrate

In [2]:
# Classify engine based on impulse

def engine_designation(imp_ns, thr_n):
    if imp_ns <= 2.5:
        impulse = "A"
    elif  imp_ns <= 5.0:
        impulse = "B"
    elif  imp_ns <= 10.0:
        impulse = "C"
    elif  imp_ns <= 20.0:
        impulse = "D"
    elif  imp_ns <= 40.0:
        impulse = "E"
    elif  imp_ns <= 80.0:
        impulse = "F"
    elif  imp_ns <= 160.0:
        impulse = "G"
    elif  imp_ns <= 320.0:
        impulse = "H"
    elif  imp_ns <= 640.0:
        impulse = "I"
    elif  imp_ns <= 1280.0:
        impulse = "J"
    elif  imp_ns <= 2560.0:
        impulse = "K"
    elif  imp_ns <= 5120.0:
        impulse = "L"
    elif  imp_ns <= 10240.0:
        impulse = "M"
    elif  imp_ns <= 20480.0:
        impulse = "N"
    elif  imp_ns <= 40960.0:
        impulse = "O"
    elif  imp_ns <= 81920.0:
        impulse = "P"
    elif  imp_ns <= 163840.0:
        impulse = "Q"
    elif  imp_ns <= 327680.0:
        impulse = "R"
    elif  imp_ns <= 655360.0:
        impulse = "S"
    else:
        impulse = "OoB"
    return impulse + str(math.floor(thr_n))

In [3]:
#https://medium.com/@krzysztofdrelczuk/time-series-anomaly-detection-with-python-example-a92ef262f09a
    
folder = "/media/paris/DataWStation/Rocketry/Measurements/Cleaned/" # Folder containing measurement files

summary_df = pd.DataFrame(columns = ["filename","Thrust_max_N","Thrust_mean_N","Impulse_Ns","Class"]) # Initialize dataframe
df_lst = []

for filename in sorted(glob.glob(folder + '*_clean.txt')):
    df = pd.read_csv(filename, sep=r';', engine='python', header=None, names=['time_s','load_g']) # Open each file as timeseries dataframe
    
    # Data processing
    df['file'] = filename[-29:-4]
    df['thrust_N'] = df['load_g'] / 1000 * 10 # Convert mass grams to kg to N
    df['time_s'] = df['time_s'] - df['time_s'].iloc[0] # Convert all timestamps to relative to recording start
    df.loc[df.thrust_N < 0, 'thrust_N'] = 0 # Replace negative values with zero
    df_lst.append(df)

    summary_df = summary_df.append({
        "filename": filename[-29:-4], 
        "Thrust_max_N": df.describe()['load_g']['max'], 
        "Thrust_mean_N": df.describe()['load_g']['mean'], 
        "Impulse_Ns": np.trapz(y=df['load_g'], x=df['time_s'])
    }, ignore_index=True)
    summary_df["Class"] = summary_df.apply(lambda x: engine_designation(x.Impulse_Ns, x.Thrust_mean_N), axis=1)

print(summary_df)
summary_df.to_csv('thrust_measurement_table.csv', index=False)

                     filename  Thrust_max_N  Thrust_mean_N   Impulse_Ns Class
0   2021-03-21_14-07-05_clean    658.441018     181.551803   776.918830  J181
1   2021-03-21_14-10-28_clean    563.194812     139.784389   930.849049  J139
2   2021-03-21_14-12-18_clean    587.199217     118.447161   675.187813  J118
3   2021-03-21_14-15-17_clean     89.838473      38.852775   276.669589   H38
4   2021-03-21_14-18-16_clean    492.607195     133.293410  1520.993343  K133
5   2021-03-21_14-21-17_clean    433.124082     168.620074  1361.582429  K168
6   2021-03-21_14-25-32_clean    191.723446      53.559606   305.874817   H53
7   2021-03-21_14-29-09_clean    551.445423     198.504669  1228.494871  J198
8   2021-03-27_15-08-18_clean    366.806412      89.951033   325.036920   I89
9   2021-03-27_15-16-30_clean    233.132893     121.262304   356.872545  I121
10  2021-03-27_15-25-38_clean    180.698483      80.000476   280.884340   H80
11  2021-03-27_15-35-58_clean     82.746451      29.934116   198

In [4]:
import pickle

with open('df_lst.pkl', 'wb') as f:
    pickle.dump(df_lst, f)

In [5]:
# Plot visualization with Bokeh

from bokeh.models import HoverTool
from bokeh.models import ColumnDataSource
from bokeh.io import output_notebook
from bokeh.layouts import gridplot
from bokeh.plotting import figure, output_file, show
output_notebook()

def color_gen():
    return (round(np.random.random()*250), round(np.random.random()*250), round(np.random.random()*250))

TOOLS = "pan, wheel_zoom, box_zoom, reset, save, box_select"
p = figure(x_axis_type="auto", tools=TOOLS, width=1280, height=720, 
           title="Thrust (N) vs Time (sec) Plot", x_axis_label='Time (sec)', y_axis_label='Thrust (N)')

for df in df_lst:
    x = df['time_s']
    y = df['thrust_N']
    source = ColumnDataSource(df)
    p.line('time_s', 'thrust_N', source=source, legend_label=df['file'][0], line_width=3, color=color_gen())
    tooltips=[("file", "@file"), ("time_s", "@time_s"), ("thrust_N", "@thrust_N")]
    p.add_tools(HoverTool(tooltips=tooltips))

output_file('thrust_graph.html')
show(p)

Loading BokehJS ...